In [1]:
# import numpy as np
# import pandas as pd
# import os
# import json
# from scipy import signal, fft
# from sklearn.decomposition import FastICA
# import traceback

# # =============================
# # CONFIGURATION
# # =============================
# BASE_PATH = '/kaggle/input/mtcaic3'
# PREPROCESSED_PATH = './quality_controlled_preprocessed/'
# EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
# MOTION_SENSORS = ['AccX', 'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
# FS = 250  # Sampling rate (Hz)
# ARTIFACT_REJECTION_THRESHOLD = 25  # Reject trials with >50% artifacts

# # =============================
# # HELPER FUNCTIONS
# # =============================
# def load_full_trial(row):
#     """Load complete trial data with all columns"""
#     try:
#         # Determine dataset split
#         id_num = row['id']
#         if id_num <= 4800:
#             split = 'train'
#         elif id_num <= 4900:
#             split = 'validation'
#         else:
#             split = 'test'
        
#         # Build file path
#         eeg_path = f"{BASE_PATH}/{row['task']}/{split}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
        
#         # Load EEG data
#         full_data = pd.read_csv(eeg_path)
        
#         # Calculate trial indices
#         trial_duration = 2250 if row['task'] == 'MI' else 1750
#         start_idx = (row['trial'] - 1) * trial_duration
#         end_idx = start_idx + trial_duration
        
#         # Handle short trials
#         if end_idx > len(full_data):
#             end_idx = len(full_data)
#             if end_idx <= start_idx:
#                 raise ValueError(f"Invalid trial range: {start_idx}-{end_idx}")
        
#         return full_data.iloc[start_idx:end_idx].copy()
    
#     except Exception as e:
#         print(f"Error loading trial {row['id']}: {str(e)}")
#         raise

# def save_preprocessed_data(preprocessed_data, row):
#     """Save preprocessed data to organized directory structure"""
#     # Determine dataset split
#     id_num = row['id']
#     if id_num <= 4800:
#         split = 'train'
#     elif id_num <= 4900:
#         split = 'validation'
#     else:
#         split = 'test'
    
#     # Create output directory
#     output_dir = f"{PREPROCESSED_PATH}/{row['task']}/{split}/{row['subject_id']}/{row['trial_session']}/"
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Save as compressed numpy file
#     output_path = f"{output_dir}/trial_{row['trial']}.npz"
#     np.savez_compressed(output_path, data=preprocessed_data.values)
    
#     return output_path

# # =============================
# # QUALITY-CONTROLLED PREPROCESSING
# # =============================
# def remove_line_noise(data, f0=50, harmonics=3):
#     """Adaptive multi-notch filtering with phase correction"""
#     filtered = data.copy()
#     n_samples = len(data)
    
#     for n in range(1, harmonics + 1):
#         freq = n * f0
#         if freq < FS/2.1:
#             Q = 35 if n == 1 else 25
#             b, a = signal.iirnotch(freq, Q, FS)
#             filtered = signal.filtfilt(b, a, filtered, axis=0)
    
#     return filtered

# def preprocess_trial(trial_data, task, trial_id=None):
#     """Quality-controlled preprocessing with artifact rejection"""
#     try:
#         # 1. Convert nV to μV
#         eeg_data = trial_data[EEG_CHANNELS].copy() / 1000
        
#         # 2. Extract validation flags and motion sensors
#         validation_flags = trial_data['Validation'].values if 'Validation' in trial_data else None
#         motion_data = trial_data[MOTION_SENSORS].copy() if all(col in trial_data for col in MOTION_SENSORS) else None
        
#         # 3. Remove DC offset using median
#         eeg_data = eeg_data - np.median(eeg_data, axis=0)
        
#         # 4. Create combined artifact mask
#         artifact_mask = np.zeros(len(eeg_data), dtype=bool)
#         artifact_counts = {'validation': 0, 'clipping': 0, 'motion': 0}
        
#         # Handle validation flags - only mark explicit invalid points
#         if validation_flags is not None:
#             validation_artifacts = (validation_flags == 0)
#             artifact_mask |= validation_artifacts
#             artifact_counts['validation'] = int(np.sum(validation_artifacts))  # Convert to int
        
#         # Optimized clipping detection
#         def detect_clipping(channel_data):
#             """Less aggressive clipping detection"""
#             # Find flat segments (consecutive identical values)
#             diff = np.diff(channel_data, prepend=channel_data[0])
#             flat_points = (np.abs(diff) < 1e-4)
            
#             # Find extreme values (only top/bottom 1% of range)
#             data_range = np.max(channel_data) - np.min(channel_data)
#             if data_range < 1e-5:  # Avoid division by zero for flat channels
#                 return np.zeros_like(channel_data, dtype=bool)
                
#             extreme_threshold = 0.01 * data_range
#             top_extreme = channel_data > (np.max(channel_data) - extreme_threshold)
#             bottom_extreme = channel_data < (np.min(channel_data) + extreme_threshold)
            
#             # Only mark as clipping if flat AND extreme
#             return flat_points & (top_extreme | bottom_extreme)
        
#         # Apply clipping detection per channel
#         clipping_mask = np.zeros(len(eeg_data), dtype=bool)
#         for channel in EEG_CHANNELS:
#             channel_mask = detect_clipping(eeg_data[channel].values)
#             clipping_mask |= channel_mask
        
#         artifact_mask |= clipping_mask
#         artifact_counts['clipping'] = int(np.sum(clipping_mask))  # Convert to int
        
#         # Motion artifact detection - less sensitive
#         if motion_data is not None and not motion_data.empty:
#             # Compute motion magnitude
#             acc_mag = np.sqrt(motion_data[['AccX', 'AccY', 'AccZ']].pow(2).sum(axis=1))
#             gyro_mag = np.sqrt(motion_data[['Gyro1', 'Gyro2', 'Gyro3']].pow(2).sum(axis=1))
#             motion_index = acc_mag + gyro_mag
            
#             # Only detect motion if there's variation
#             if motion_index.nunique() > 1:
#                 # Use percentiles instead of z-scores
#                 upper_quartile = np.percentile(motion_index, 75)
#                 lower_quartile = np.percentile(motion_index, 25)
#                 iqr = upper_quartile - lower_quartile
                
#                 # Higher threshold for motion detection
#                 motion_threshold = upper_quartile + 3 * iqr
#                 motion_artifacts = motion_index > motion_threshold
                
#                 artifact_mask |= motion_artifacts
#                 artifact_counts['motion'] = int(np.sum(motion_artifacts))  # Convert to int
        
#         # Calculate artifact percentage
#         total_artifacts = np.sum(artifact_mask)
#         artifact_percent = total_artifacts / len(artifact_mask) * 100
        
#         # ===== QUALITY CONTROL =====
#         # Reject severely contaminated trials
#         if artifact_percent > ARTIFACT_REJECTION_THRESHOLD:
#             if trial_id:
#                 print(f"REJECTING trial {trial_id}: {artifact_percent:.1f}% artifacts")
#             return None, artifact_percent, artifact_counts
        
#         # Log artifact statistics
#         if trial_id:
#             print(f"Trial {trial_id}: Artifacts - {artifact_percent:.1f}% "
#                   f"(Validation: {artifact_counts['validation']}, "
#                   f"Clipping: {artifact_counts['clipping']}, "
#                   f"Motion: {artifact_counts['motion']})")
        
#         # Expand artifact regions conservatively
#         expanded_mask = artifact_mask.copy()
#         artifact_indices = np.where(artifact_mask)[0]
        
#         if len(artifact_indices) > 0:
#             # Only expand by 50ms (12 samples) instead of 100ms
#             for i in artifact_indices:
#                 start = max(0, i - 12)
#                 end = min(len(artifact_mask), i + 13)
#                 expanded_mask[start:end] = True
        
#         # 5. Apply artifact mask with validation
#         valid_mask = ~expanded_mask
#         x = np.arange(len(eeg_data))
        
#         # Ensure there are enough valid points
#         min_valid_points = max(50, int(0.1 * len(eeg_data)))  # At least 50 points or 10% of trial
        
#         if np.sum(valid_mask) < min_valid_points:
#             # Use fallback: mark as valid unless explicitly invalid
#             valid_mask = ~artifact_mask  # Only original artifacts
#             if np.sum(valid_mask) < min_valid_points:
#                 valid_mask = np.ones(len(eeg_data), dtype=bool)
#                 if trial_id:
#                     print(f"Warning: Limited valid points in trial {trial_id}. Using all points")
        
#         # Apply interpolation per channel
#         for channel in EEG_CHANNELS:
#             channel_data = eeg_data[channel].values
#             eeg_data[channel] = np.interp(x, x[valid_mask], channel_data[valid_mask])
        
#         # 6. Enhanced Line Noise Removal
#         eeg_values = remove_line_noise(eeg_data.values)
#         eeg_data = pd.DataFrame(eeg_values, columns=EEG_CHANNELS)
        
#         # 7. ERD Preservation
#         sos_bp = signal.butter(3, [4, 40], btype='bandpass', output='sos', fs=FS)
#         filtered = signal.sosfiltfilt(sos_bp, eeg_data.values, axis=0)
        
#         # 8. Motion artifact handling with ICA (only when significant motion artifacts detected)
#         if motion_data is not None and not motion_data.empty and artifact_counts['motion'] > 50:
#             try:
#                 ica = FastICA(n_components=len(EEG_CHANNELS), random_state=0, max_iter=500)
#                 ica_sources = ica.fit_transform(filtered)
                
#                 # Compute motion index
#                 acc_mag = np.sqrt(motion_data[['AccX', 'AccY', 'AccZ']].pow(2).sum(axis=1))
#                 gyro_mag = np.sqrt(motion_data[['Gyro1', 'Gyro2', 'Gyro3']].pow(2).sum(axis=1))
#                 motion_index = acc_mag + gyro_mag
                
#                 # Identify motion-correlated components
#                 motion_corr = np.zeros(ica_sources.shape[1])
#                 for i in range(ica_sources.shape[1]):
#                     corr = np.corrcoef(np.abs(ica_sources[:, i]), np.abs(motion_index))[0, 1]
#                     motion_corr[i] = np.abs(corr)
                
#                 # Remove only strongly correlated components
#                 motion_components = motion_corr > 0.7
#                 if np.any(motion_components):
#                     ica_sources[:, motion_components] = 0
#                     filtered = ica.inverse_transform(ica_sources)
#             except Exception as e:
#                 if trial_id:
#                     print(f"ICA skipped in trial {trial_id}: {str(e)}")
        
#         # 9. Final output
#         eeg_data = pd.DataFrame(filtered, columns=EEG_CHANNELS)
        
#         return eeg_data, artifact_percent, artifact_counts
    
#     except Exception as e:
#         print(f"Error during preprocessing trial {trial_id}: {str(e)}")
#         traceback.print_exc()
#         return None, None, None

# # =============================
# # PIPELINE EXECUTION WITH QUALITY TRACKING
# # =============================
# def preprocess_dataset(df, dataset_name):
#     """Preprocess entire dataset with quality tracking"""
#     print(f"Preprocessing {dataset_name} set with {len(df)} trials...")
    
#     processed_paths = []
#     artifact_percentages = []
#     artifact_reports = []
#     rejected_trials = []
#     error_trials = []
    
#     for idx, row in df.iterrows():
#         trial_id = int(row['id'])  # Convert to native Python int
#         try:
#             # Load full trial data
#             trial_data = load_full_trial(row)
            
#             # Apply preprocessing pipeline
#             preprocessed, artifact_percent, artifact_counts = preprocess_trial(
#                 trial_data, row['task'], trial_id=trial_id
#             )
            
#             # Create artifact report
#             report_entry = {
#                 'trial_id': trial_id,
#             }
            
#             # Track artifact metrics
#             if artifact_percent is not None:
#                 artifact_percentages.append(float(artifact_percent))  # Convert to float
#                 report_entry['artifact_percent'] = float(artifact_percent)
            
#             if artifact_counts is not None:
#                 # Convert counts to native Python integers
#                 report_entry['artifact_counts'] = {k: int(v) for k, v in artifact_counts.items()}
            
#             artifact_reports.append(report_entry)
            
#             # Handle rejected trials
#             if preprocessed is None and artifact_percent is not None:
#                 processed_paths.append(None)
#                 rejected_trials.append(trial_id)
#                 continue
                
#             # Handle processing errors
#             if preprocessed is None:
#                 processed_paths.append(None)
#                 error_trials.append(trial_id)
#                 continue
                
#             # Save processed data
#             output_path = save_preprocessed_data(preprocessed, row)
#             processed_paths.append(output_path)
            
#             # Print progress
#             if (idx + 1) % 100 == 0:
#                 print(f"Processed {idx+1}/{len(df)} trials")
                
#         except Exception as e:
#             print(f"Error processing trial {trial_id}: {str(e)}")
#             processed_paths.append(None)
#             error_trials.append(trial_id)
#             artifact_reports.append({
#                 'trial_id': trial_id,
#                 'error': str(e)
#             })
    
#     # Add paths to dataframe
#     df['processed_path'] = processed_paths
    
#     # Save updated dataframe
#     df.to_csv(f"{PREPROCESSED_PATH}/{dataset_name}.csv", index=False)
    
#     # Calculate statistics
#     processed_count = len([p for p in processed_paths if p is not None])
#     rejected_count = len(rejected_trials)
#     error_count = len(error_trials)
#     valid_artifact_percentages = [p for p in artifact_percentages if not np.isnan(p)]
    
#     # Save quality report
#     quality_report = {
#         'dataset': dataset_name,
#         'total_trials': len(df),
#         'processed_trials': processed_count,
#         'rejected_trials': rejected_trials,
#         'error_trials': error_trials,
#         'rejection_rate': rejected_count / len(df) * 100,
#         'error_rate': error_count / len(df) * 100,
#         'mean_artifact_percent': float(np.mean(valid_artifact_percentages)) if valid_artifact_percentages else 0,
#         'median_artifact_percent': float(np.median(valid_artifact_percentages)) if valid_artifact_percentages else 0,
#         'artifact_reports': artifact_reports
#     }
    
#     # Save with native Python types
#     with open(f"{PREPROCESSED_PATH}/{dataset_name}_quality.json", 'w') as f:
#         json.dump(quality_report, f, indent=2, default=str)
    
#     # Print summary
#     print(f"\n===== {dataset_name.upper()} SET SUMMARY =====")
#     print(f"Total trials: {len(df)}")
#     print(f"Successfully processed: {processed_count} ({processed_count/len(df)*100:.1f}%)")
#     print(f"Rejected trials: {rejected_count} ({rejected_count/len(df)*100:.1f}%)")
#     print(f"Error trials: {error_count} ({error_count/len(df)*100:.1f}%)")
#     if valid_artifact_percentages:
#         print(f"Average artifact percentage: {np.mean(valid_artifact_percentages):.1f}%")
#         print(f"Median artifact percentage: {np.median(valid_artifact_percentages):.1f}%")
    
#     return df

# # =============================
# # MAIN EXECUTION
# # =============================
# if __name__ == "__main__":
#     # Create output directory
#     os.makedirs(PREPROCESSED_PATH, exist_ok=True)
    
#     # Load datasets
#     train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
#     val_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))
#     test_df = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
    
#     # Preprocess all datasets
#     print("Starting quality-controlled preprocessing...")
#     print(f"Rejection threshold: >{ARTIFACT_REJECTION_THRESHOLD}% artifacts\n")
    
#     print("=== PROCESSING TRAINING SET ===")
#     train_df = preprocess_dataset(train_df, 'train')
    
#     print("\n=== PROCESSING VALIDATION SET ===")
#     val_df = preprocess_dataset(val_df, 'validation')
    
#     print("\n=== PROCESSING TEST SET ===")
#     test_df = preprocess_dataset(test_df, 'test')
    
#     # Save final metadata
#     train_df.to_csv(os.path.join(PREPROCESSED_PATH, 'train.csv'), index=False)
#     val_df.to_csv(os.path.join(PREPROCESSED_PATH, 'validation.csv'), index=False)
#     test_df.to_csv(os.path.join(PREPROCESSED_PATH, 'test.csv'), index=False)
    
#     # Final summary
#     total_trials = len(train_df) + len(val_df) + len(test_df)
#     total_processed = (
#         sum(1 for p in train_df['processed_path'] if p is not None) +
#         sum(1 for p in val_df['processed_path'] if p is not None) +
#         sum(1 for p in test_df['processed_path'] if p is not None)
#     )
    
#     print("\n===== FINAL SUMMARY =====")
#     print(f"Total trials in dataset: {total_trials}")
#     print(f"Successfully processed: {total_processed} ({total_processed/total_trials*100:.1f}%)")
#     print(f"Rejection threshold: >{ARTIFACT_REJECTION_THRESHOLD}% artifacts")
#     print("\nPreprocessing complete! Data saved to:", PREPROCESSED_PATH)
#     print("Quality reports available for each dataset:")
#     print("- train_quality.json")
#     print("- validation_quality.json")
#     print("- test_quality.json")
    
#     print("\nTo analyze results:")
#     print("import json, pandas as pd")
#     print("report = json.load(open('./quality_controlled_preprocessed/train_quality.json'))")
#     print("print(f\"Rejected trials: {len(report['rejected_trials'])}\")")
#     print("print(f\"Error trials: {len(report['error_trials'])}\")")

In [2]:
import numpy as np
import pandas as pd
import os
import json
from scipy import signal, fft
from sklearn.decomposition import FastICA
import traceback

# =============================
# CONFIGURATION
# =============================
BASE_PATH = '/kaggle/input/mtcaic3'
PREPROCESSED_PATH = './quality_controlled_preprocessed/'
EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
MOTION_SENSORS = ['AccX', 'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
FS = 250  # Sampling rate (Hz)
ARTIFACT_REJECTION_THRESHOLD = 25  # Reject trials with >50% artifacts

# =============================
# HELPER FUNCTIONS
# =============================
def load_full_trial(row):
    """Load complete trial data with all columns"""
    try:
        # Determine dataset split
        id_num = row['id']
        if id_num <= 4800:
            split = 'train'
        elif id_num <= 4900:
            split = 'validation'
        else:
            split = 'test'
        
        # Build file path
        eeg_path = f"{BASE_PATH}/{row['task']}/{split}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
        
        # Load EEG data
        full_data = pd.read_csv(eeg_path)
        
        # Calculate trial indices
        trial_duration = 2250 if row['task'] == 'MI' else 1750
        start_idx = (row['trial'] - 1) * trial_duration
        end_idx = start_idx + trial_duration
        
        # Handle short trials
        if end_idx > len(full_data):
            end_idx = len(full_data)
            if end_idx <= start_idx:
                raise ValueError(f"Invalid trial range: {start_idx}-{end_idx}")
        
        return full_data.iloc[start_idx:end_idx].copy()
    
    except Exception as e:
        print(f"Error loading trial {row['id']}: {str(e)}")
        raise

def save_preprocessed_data(preprocessed_data, row):
    """Save preprocessed data to organized directory structure as CSV"""
    # Determine dataset split
    id_num = row['id']
    if id_num <= 4800:
        split = 'train'
    elif id_num <= 4900:
        split = 'validation'
    else:
        split = 'test'
    
    # Create output directory
    output_dir = f"{PREPROCESSED_PATH}/{row['task']}/{split}/{row['subject_id']}/{row['trial_session']}/"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save as CSV file
    output_path = f"{output_dir}/trial_{row['trial']}.csv"
    preprocessed_data.to_csv(output_path, index=False)
    
    return output_path

# =============================
# QUALITY-CONTROLLED PREPROCESSING
# =============================
def remove_line_noise(data, f0=50, harmonics=3):
    """Adaptive multi-notch filtering with phase correction"""
    filtered = data.copy()
    n_samples = len(data)
    
    for n in range(1, harmonics + 1):
        freq = n * f0
        if freq < FS/2.1:
            Q = 35 if n == 1 else 25
            b, a = signal.iirnotch(freq, Q, FS)
            filtered = signal.filtfilt(b, a, filtered, axis=0)
    
    return filtered

def preprocess_trial(trial_data, task, trial_id=None):
    """Quality-controlled preprocessing with artifact rejection"""
    try:
        # 1. Convert nV to μV
        eeg_data = trial_data[EEG_CHANNELS].copy() / 1000
        
        # 2. Extract validation flags and motion sensors
        validation_flags = trial_data['Validation'].values if 'Validation' in trial_data else None
        motion_data = trial_data[MOTION_SENSORS].copy() if all(col in trial_data for col in MOTION_SENSORS) else None
        
        # 3. Remove DC offset using median
        eeg_data = eeg_data - np.median(eeg_data, axis=0)
        
        # 4. Create combined artifact mask
        artifact_mask = np.zeros(len(eeg_data), dtype=bool)
        artifact_counts = {'validation': 0, 'clipping': 0, 'motion': 0}
        
        # Handle validation flags - only mark explicit invalid points
        if validation_flags is not None:
            validation_artifacts = (validation_flags == 0)
            artifact_mask |= validation_artifacts
            artifact_counts['validation'] = int(np.sum(validation_artifacts))  # Convert to int
        
        # Optimized clipping detection
        def detect_clipping(channel_data):
            """Less aggressive clipping detection"""
            # Find flat segments (consecutive identical values)
            diff = np.diff(channel_data, prepend=channel_data[0])
            flat_points = (np.abs(diff) < 1e-4)
            
            # Find extreme values (only top/bottom 1% of range)
            data_range = np.max(channel_data) - np.min(channel_data)
            if data_range < 1e-5:  # Avoid division by zero for flat channels
                return np.zeros_like(channel_data, dtype=bool)
                
            extreme_threshold = 0.01 * data_range
            top_extreme = channel_data > (np.max(channel_data) - extreme_threshold)
            bottom_extreme = channel_data < (np.min(channel_data) + extreme_threshold)
            
            # Only mark as clipping if flat AND extreme
            return flat_points & (top_extreme | bottom_extreme)
        
        # Apply clipping detection per channel
        clipping_mask = np.zeros(len(eeg_data), dtype=bool)
        for channel in EEG_CHANNELS:
            channel_mask = detect_clipping(eeg_data[channel].values)
            clipping_mask |= channel_mask
        
        artifact_mask |= clipping_mask
        artifact_counts['clipping'] = int(np.sum(clipping_mask))  # Convert to int
        
        # Motion artifact detection - less sensitive
        if motion_data is not None and not motion_data.empty:
            # Compute motion magnitude
            acc_mag = np.sqrt(motion_data[['AccX', 'AccY', 'AccZ']].pow(2).sum(axis=1))
            gyro_mag = np.sqrt(motion_data[['Gyro1', 'Gyro2', 'Gyro3']].pow(2).sum(axis=1))
            motion_index = acc_mag + gyro_mag
            
            # Only detect motion if there's variation
            if motion_index.nunique() > 1:
                # Use percentiles instead of z-scores
                upper_quartile = np.percentile(motion_index, 75)
                lower_quartile = np.percentile(motion_index, 25)
                iqr = upper_quartile - lower_quartile
                
                # Higher threshold for motion detection
                motion_threshold = upper_quartile + 3 * iqr
                motion_artifacts = motion_index > motion_threshold
                
                artifact_mask |= motion_artifacts
                artifact_counts['motion'] = int(np.sum(motion_artifacts))  # Convert to int
        
        # Calculate artifact percentage
        total_artifacts = np.sum(artifact_mask)
        artifact_percent = total_artifacts / len(artifact_mask) * 100
        
        # ===== QUALITY CONTROL =====
        # Reject severely contaminated trials
        if artifact_percent > ARTIFACT_REJECTION_THRESHOLD:
            if trial_id:
                print(f"REJECTING trial {trial_id}: {artifact_percent:.1f}% artifacts")
            return None, artifact_percent, artifact_counts
        
        # Log artifact statistics
        if trial_id:
            print(f"Trial {trial_id}: Artifacts - {artifact_percent:.1f}% "
                  f"(Validation: {artifact_counts['validation']}, "
                  f"Clipping: {artifact_counts['clipping']}, "
                  f"Motion: {artifact_counts['motion']})")
        
        # Expand artifact regions conservatively
        expanded_mask = artifact_mask.copy()
        artifact_indices = np.where(artifact_mask)[0]
        
        if len(artifact_indices) > 0:
            # Only expand by 50ms (12 samples) instead of 100ms
            for i in artifact_indices:
                start = max(0, i - 12)
                end = min(len(artifact_mask), i + 13)
                expanded_mask[start:end] = True
        
        # 5. Apply artifact mask with validation
        valid_mask = ~expanded_mask
        x = np.arange(len(eeg_data))
        
        # Ensure there are enough valid points
        min_valid_points = max(50, int(0.1 * len(eeg_data)))  # At least 50 points or 10% of trial
        
        if np.sum(valid_mask) < min_valid_points:
            # Use fallback: mark as valid unless explicitly invalid
            valid_mask = ~artifact_mask  # Only original artifacts
            if np.sum(valid_mask) < min_valid_points:
                valid_mask = np.ones(len(eeg_data), dtype=bool)
                if trial_id:
                    print(f"Warning: Limited valid points in trial {trial_id}. Using all points")
        
        # Apply interpolation per channel
        for channel in EEG_CHANNELS:
            channel_data = eeg_data[channel].values
            eeg_data[channel] = np.interp(x, x[valid_mask], channel_data[valid_mask])
        
        # 6. Enhanced Line Noise Removal
        eeg_values = remove_line_noise(eeg_data.values)
        eeg_data = pd.DataFrame(eeg_values, columns=EEG_CHANNELS)
        
        # 7. ERD Preservation
        sos_bp = signal.butter(3, [4, 40], btype='bandpass', output='sos', fs=FS)
        filtered = signal.sosfiltfilt(sos_bp, eeg_data.values, axis=0)
        
        # 8. Motion artifact handling with ICA (only when significant motion artifacts detected)
        if motion_data is not None and not motion_data.empty and artifact_counts['motion'] > 50:
            try:
                ica = FastICA(n_components=len(EEG_CHANNELS), random_state=0, max_iter=500)
                ica_sources = ica.fit_transform(filtered)
                
                # Compute motion index
                acc_mag = np.sqrt(motion_data[['AccX', 'AccY', 'AccZ']].pow(2).sum(axis=1))
                gyro_mag = np.sqrt(motion_data[['Gyro1', 'Gyro2', 'Gyro3']].pow(2).sum(axis=1))
                motion_index = acc_mag + gyro_mag
                
                # Identify motion-correlated components
                motion_corr = np.zeros(ica_sources.shape[1])
                for i in range(ica_sources.shape[1]):
                    corr = np.corrcoef(np.abs(ica_sources[:, i]), np.abs(motion_index))[0, 1]
                    motion_corr[i] = np.abs(corr)
                
                # Remove only strongly correlated components
                motion_components = motion_corr > 0.7
                if np.any(motion_components):
                    ica_sources[:, motion_components] = 0
                    filtered = ica.inverse_transform(ica_sources)
            except Exception as e:
                if trial_id:
                    print(f"ICA skipped in trial {trial_id}: {str(e)}")
        
        # 9. Final output
        eeg_data = pd.DataFrame(filtered, columns=EEG_CHANNELS)
        
        return eeg_data, artifact_percent, artifact_counts
    
    except Exception as e:
        print(f"Error during preprocessing trial {trial_id}: {str(e)}")
        traceback.print_exc()
        return None, None, None

# =============================
# PIPELINE EXECUTION WITH QUALITY TRACKING
# =============================
def preprocess_dataset(df, dataset_name):
    """Preprocess entire dataset with quality tracking"""
    print(f"Preprocessing {dataset_name} set with {len(df)} trials...")
    
    processed_paths = []
    artifact_percentages = []
    artifact_reports = []
    rejected_trials = []
    error_trials = []
    
    for idx, row in df.iterrows():
        trial_id = int(row['id'])  # Convert to native Python int
        try:
            # Load full trial data
            trial_data = load_full_trial(row)
            
            # Apply preprocessing pipeline
            preprocessed, artifact_percent, artifact_counts = preprocess_trial(
                trial_data, row['task'], trial_id=trial_id
            )
            
            # Create artifact report
            report_entry = {
                'trial_id': trial_id,
            }
            
            # Track artifact metrics
            if artifact_percent is not None:
                artifact_percentages.append(float(artifact_percent))  # Convert to float
                report_entry['artifact_percent'] = float(artifact_percent)
            
            if artifact_counts is not None:
                # Convert counts to native Python integers
                report_entry['artifact_counts'] = {k: int(v) for k, v in artifact_counts.items()}
            
            artifact_reports.append(report_entry)
            
            # Handle rejected trials
            if preprocessed is None and artifact_percent is not None:
                processed_paths.append(None)
                rejected_trials.append(trial_id)
                continue
                
            # Handle processing errors
            if preprocessed is None:
                processed_paths.append(None)
                error_trials.append(trial_id)
                continue
                
            # Save processed data
            output_path = save_preprocessed_data(preprocessed, row)
            processed_paths.append(output_path)
            
            # Print progress
            if (idx + 1) % 100 == 0:
                print(f"Processed {idx+1}/{len(df)} trials")
                
        except Exception as e:
            print(f"Error processing trial {trial_id}: {str(e)}")
            processed_paths.append(None)
            error_trials.append(trial_id)
            artifact_reports.append({
                'trial_id': trial_id,
                'error': str(e)
            })
    
    # Add paths to dataframe
    df['processed_path'] = processed_paths
    
    # Save updated dataframe
    df.to_csv(f"{PREPROCESSED_PATH}/{dataset_name}.csv", index=False)
    
    # Calculate statistics
    processed_count = len([p for p in processed_paths if p is not None])
    rejected_count = len(rejected_trials)
    error_count = len(error_trials)
    valid_artifact_percentages = [p for p in artifact_percentages if not np.isnan(p)]
    
    # Save quality report
    quality_report = {
        'dataset': dataset_name,
        'total_trials': len(df),
        'processed_trials': processed_count,
        'rejected_trials': rejected_trials,
        'error_trials': error_trials,
        'rejection_rate': rejected_count / len(df) * 100,
        'error_rate': error_count / len(df) * 100,
        'mean_artifact_percent': float(np.mean(valid_artifact_percentages)) if valid_artifact_percentages else 0,
        'median_artifact_percent': float(np.median(valid_artifact_percentages)) if valid_artifact_percentages else 0,
        'artifact_reports': artifact_reports
    }
    
    # Save with native Python types
    with open(f"{PREPROCESSED_PATH}/{dataset_name}_quality.json", 'w') as f:
        json.dump(quality_report, f, indent=2, default=str)
    
    # Print summary
    print(f"\n===== {dataset_name.upper()} SET SUMMARY =====")
    print(f"Total trials: {len(df)}")
    print(f"Successfully processed: {processed_count} ({processed_count/len(df)*100:.1f}%)")
    print(f"Rejected trials: {rejected_count} ({rejected_count/len(df)*100:.1f}%)")
    print(f"Error trials: {error_count} ({error_count/len(df)*100:.1f}%)")
    if valid_artifact_percentages:
        print(f"Average artifact percentage: {np.mean(valid_artifact_percentages):.1f}%")
        print(f"Median artifact percentage: {np.median(valid_artifact_percentages):.1f}%")
    
    return df

# =============================
# MAIN EXECUTION
# =============================
if __name__ == "__main__":
    # Create output directory
    os.makedirs(PREPROCESSED_PATH, exist_ok=True)
    
    # Load datasets
    train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
    val_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))
    test_df = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
    
    # Preprocess all datasets
    print("Starting quality-controlled preprocessing...")
    print(f"Rejection threshold: >{ARTIFACT_REJECTION_THRESHOLD}% artifacts\n")
    
    print("=== PROCESSING TRAINING SET ===")
    train_df = preprocess_dataset(train_df, 'train')
    
    print("\n=== PROCESSING VALIDATION SET ===")
    val_df = preprocess_dataset(val_df, 'validation')
    
    print("\n=== PROCESSING TEST SET ===")
    test_df = preprocess_dataset(test_df, 'test')
    
    # Save final metadata
    train_df.to_csv(os.path.join(PREPROCESSED_PATH, 'train.csv'), index=False)
    val_df.to_csv(os.path.join(PREPROCESSED_PATH, 'validation.csv'), index=False)
    test_df.to_csv(os.path.join(PREPROCESSED_PATH, 'test.csv'), index=False)
    
    # Final summary
    total_trials = len(train_df) + len(val_df) + len(test_df)
    total_processed = (
        sum(1 for p in train_df['processed_path'] if p is not None) +
        sum(1 for p in val_df['processed_path'] if p is not None) +
        sum(1 for p in test_df['processed_path'] if p is not None)
    )
    
    print("\n===== FINAL SUMMARY =====")
    print(f"Total trials in dataset: {total_trials}")
    print(f"Successfully processed: {total_processed} ({total_processed/total_trials*100:.1f}%)")
    print(f"Rejection threshold: >{ARTIFACT_REJECTION_THRESHOLD}% artifacts")
    print("\nPreprocessing complete! Data saved to:", PREPROCESSED_PATH)
    print("Quality reports available for each dataset:")
    print("- train_quality.json")
    print("- validation_quality.json")
    print("- test_quality.json")
    
    print("\nTo analyze results:")
    print("import json, pandas as pd")
    print("report = json.load(open('./quality_controlled_preprocessed/train_quality.json'))")
    print("print(f\"Rejected trials: {len(report['rejected_trials'])}\")")
    print("print(f\"Error trials: {len(report['error_trials'])}\")")

Starting quality-controlled preprocessing...
Rejection threshold: >25% artifacts

=== PROCESSING TRAINING SET ===
Preprocessing train set with 4800 trials...
Trial 1: Artifacts - 7.4% (Validation: 72, Clipping: 0, Motion: 95)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Trial 2: Artifacts - 0.0% (Validation: 0, Clipping: 0, Motion: 0)
Trial 3: Artifacts - 0.0% (Validation: 0, Clipping: 0, Motion: 0)
Trial 4: Artifacts - 0.0% (Validation: 0, Clipping: 0, Motion: 0)
Trial 5: Artifacts - 0.0% (Validation: 0, Clipping: 0, Motion: 0)
Trial 6: Artifacts - 4.9% (Validation: 70, Clipping: 0, Motion: 95)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Trial 7: Artifacts - 6.0% (Validation: 0, Clipping: 0, Motion: 134)


...

Trial 4997: Artifacts - 10.8% (Validation: 0, Clipping: 0, Motion: 189)
Trial 4998: Artifacts - 13.8% (Validation: 0, Clipping: 1, Motion: 240)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Trial 4999: Artifacts - 10.1% (Validation: 0, Clipping: 0, Motion: 177)
Trial 5000: Artifacts - 11.7% (Validation: 0, Clipping: 0, Motion: 205)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Processed 100/100 trials

===== TEST SET SUMMARY =====
Total trials: 100
Successfully processed: 100 (100.0%)
Rejected trials: 0 (0.0%)
Error trials: 0 (0.0%)
Average artifact percentage: 5.0%
Median artifact percentage: 3.7%

===== FINAL SUMMARY =====
Total trials in dataset: 5000
Successfully processed: 4829 (96.6%)
Rejection threshold: >25% artifacts

Preprocessing complete! Data saved to: ./quality_controlled_preprocessed/
Quality reports available for each dataset:
- train_quality.json
- validation_quality.json
- test_quality.json

To analyze results:
import json, pandas as pd
report = json.load(open('./quality_controlled_preprocessed/train_quality.json'))
print(f"Rejected trials: {len(report['rejected_trials'])}")
print(f"Error trials: {len(report['error_trials'])}")
